# Search Script Scrape

Search Script Scrape was created by Dan Nguyen, to learn his students at Stanford Journalism College how to scrape by writing 101 scrapers. Below you'll find the scrapers as written by me, Winny de Jong. Some scrapers are written multiple times using different libraries/techniques. Just because. 

All scrapers worked at a certain point in time; from scraper 009 onwards, I've added this certain point of time. :) 

- [The original Github Repository](https://github.com/stanfordjournalism/search-script-scrape)
- Article by Dan Nguyen [with background info](http://blog.danwin.com/examples-of-web-scraping-in-python-3-x-for-data-journalists/)

## 001, v1

Scraper 001 using requests and lxml  
Goal: get the number of datasets currently listed on [data.gov](http://www.data.gov/)


In [ ]:
# import needed libraries
from lxml import html
import requests

# import site by get request in variable site1
site = requests.get('http://www.data.gov/')

# save html as text in variable doc
doc = html.fromstring(site.text)

# select the first [0] link found by selecting css; save it as ds1
datasets = doc.cssselect('small a')[0]

# print result
print(datasets.text.replace(',', ''))

## 001, v2

Scraper 001 - using beautifulsoup and requests  
Goal:  get the number of datasets currently listed on [data.gov](http://www.data.gov/)

In [ ]:
# import needed libraries
import requests
from bs4 import BeautifulSoup

# import site by get request in variable site1
site = requests.get('http://www.data.gov/')

# make bs object from website
soup = BeautifulSoup(site.content, 'html.parser')

# save content div with class new-results in datasets
datasets = soup.find('small')

# save datasets as text in variable dt
dt = datasets.text

# slice dt to only get the number
dt[12:19]

## 002, v1

Scraper 002 - using requests and lxml and css select  
Goal: get the name of the [most recently added dataset on data.gov](https://catalog.data.gov/dataset?q=&sort=metadata_created+desc)

In [ ]:
# import needed libraries
from lxml import html
import requests

# import site by get request in variable site1
site = requests.get('https://catalog.data.gov/dataset?q=&sort=metadata_created+desc')

# save html as text in variable doc
doc = html.fromstring(site.text)

# select the first [0] link found by selecting css; save it as ds1
ds1 = doc.cssselect('h3')[0].text_content()

# print result
print(ds1)

## 002, v2

Scraper 002 - using requests and lxml and css select  
Goal: get the name of the [most recently added dataset on data.gov](https://catalog.data.gov/dataset?q=&sort=metadata_created+desc)

In [ ]:
# import needed libraries
from bs4 import BeautifulSoup
import requests

# create a variable with the url
url = 'https://catalog.data.gov/dataset?q=&sort=metadata_created+desc'

# Use requests to get the contents
r = requests.get(url)

# get the text of the contents
html_content = r.text

# convert the html content into a beautiful soup object
soup = BeautifulSoup(html_content, 'lxml')

# get text of first h3 heading
soup.h3.a.get_text()

## 003

Scraper 003, using requests and navigating a json file  
Goal: get the number of [people who visited a U.S. government website](https://analytics.usa.gov/data/live/ie.json) using Internet Explorer 6.0 in the last 90 days   
Written april 8th, 2018.

In [ ]:
import requests

r = requests.get("https://analytics.usa.gov/data/live/ie.json")
# read json file with .json(), select info totals > ie_version > 6.0
ie6 = (r.json()['totals']['ie_version']['6.0'])
print(ie6)

## Skipped 004

Scraper 004 - using using requests and navigating a json file  
Goal: get the number of librarian-related job positions that the federal government is currently hiring for, code for librarian-related positions is 1410.  
Written april 8th, 2018.  
Given url + api don't seem to work. (https://data.usajobs.gov/api/jobs?series=1410)

## TO DO 005

Scraper 005, using ...  
Goal: get the name of the company cited in the most recent consumer complaint involving student loans 

## 006

Scraper 006, using requests and beautiful soup  
Goal: get the the change in median cost of health, dental, and vision coverage for California city employees from 2010 to 2013.

Since the provided URL gave me errors, I opted for [transparent California](https://transparentcalifornia.com/salaries/2016/california-city/) as an alternative. I'm looking up the median total benefits for employees of California City from 2011 to 2014.
  
Written december 24th 2019.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://transparentcalifornia.com/salaries/XXX/california-city/"
data = []

for i in range(2011,2015):
    i = str(i)
    url = url.replace("XXX", i)
    r = requests.get(url)
    # get the text of the contents
    html_content = r.text
    # convert the html content into a beautiful soup object
    soup = BeautifulSoup(html_content, 'html.parser')
    # get the div that contains the pagination
    divPages = soup.find('div', {'class': 'pagination pagination-centered'})
    # find all list items of the pagination
    pages = divPages.find_all('li')
    # since the pagination ends with 'Next >', the second last 
    # item contains the highest pagenumber
    maxPage = len(pages) - 2
    
    # for every page in pagerange:
    for page in range(0, maxPage):
        # request page...
        url = url + '?page=' + str(page)
        r = requests.get(url)
        # get the text of the contents
        html_content = r.text
        # convert the html content into a beautiful soup object
        soup = BeautifulSoup(html_content, 'html.parser')
        # get all rows froms table
        for t in soup.find_all('tr'):
            cells = t.find_all('td')
            try:
                year = i
                # get name from first td
                name = cells[0].text.strip()
                # get benefits from 7th td, replace $ and ,
                benefits = cells[6].text.replace('$', '').replace(',', '')
                # append information to list with data
                data.append([year, name, benefits])
            except:
                # this exception is necessary since some tr's do not contain any
                # cells, but instead are used to show an ad-banner. We want to simply
                # pass those banner containing tr's...
                pass
    
# NOTE: get out of for-loops = no more indentation

# create a dataframe based upon data
df = pd.DataFrame(data)
# set column names
df.columns = ['year', 'name', 'benefits']
# create empty list to later collect medians in
medians = []
# calculate median benefits for every year
for year in df['year'].unique():
    # create sub dataframe
    subDF = df[df['year'] == year]
    subDF
    # calculate median benefits for subDF
    median = subDF['benefits'].median()
    # append data to list of medians
    medians.append([year, median])

# create dataframe based on list of lists
medianDF = pd.DataFrame(medians)
# rename columns
medianDF.columns = ['year', 'median']
# return dataframe medianDF
medianDF

## 007

The [URL provided](https://inventory.data.gov/dataset/federal-executive-agency-internet-domains-as-of-06172014/resource/b626ef1f-9019-41c4-91aa-5ae3f7457328) is no longer valid, alternative URL's require a login.  
Goal: get the number of listed federal executive agency internet domains 

## 008, v1

Scraper 008 using requests and pandas and a url that leads to a CSV-file.  
Goal: get the number of times when a New York heart surgeon's rate of patient deaths for all cardiac surgical procedures was "significantly higher" than the statewide rate, according to New York state's analysis.

In [ ]:
import requests
import pandas as pd

url = "https://health.data.ny.gov/resource/msvb-mnxf.csv"
data = pd.read_csv(url)
records = data.loc[data['comparison_results'] == 'Rate significantly higher than Statewide Rate']
print(len(records))

## 008, v2

Scraper 008 using requests and a url that leads to a JSON-file.   
Goal: get the number of times when a New York heart surgeon's rate of patient deaths for all cardiac surgical procedures was "significantly higher" than the statewide rate, according to New York state's analysis.

In [ ]:
import requests

url = 'https://health.data.ny.gov/resource/dk4z-k3xb.json'
string = 'Rate significantly higher than Statewide Rate'
data = requests.get(url).json()
records = [r for r in data if string in r['comparison_results']]
print(len(records))

## 009

Scraper 009, using ....   
Goal: get the number of [roll call votes](https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_114_1.htm) that were rejected by a margin of less than 5 votes, in the first session of the U.S. Senate in the 114th Congress .

In [ ]:
import requests
from lxml import etree

url = "https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_114_1.xml"

data = requests.get(url)

tree = etree.fromstring(data.content)
try: 
    result = tree.xpath('child::*/child::result')
except:
    result = ''
print(result)

## 010

Scraper 010, using requests and beautiful soup.  
Goal: get the title of the highest paid California city government position in 2010.

In [ ]:
import requests
from bs4 import BeautifulSoup

# a Google search gave me this url
url = 'https://publicpay.ca.gov/Reports/Cities/Cities.aspx?year=2010&rpt=1'
# a look at the source code of that url, showed me that the actual request
# takes place elsewhere, namely:
dataUrl = 'https://publicpay.ca.gov/Ajax/Reports/GetRptTopEmployees.aspx?year=2010&entitytypeid=1&rptfilter=0&showfilter=True'
# request website
r = requests.get(dataUrl)
# create soup
soup = BeautifulSoup(r.content)
# get all jobs from table
jobs = soup.find_all('tr')

# get highestPaid looking nicely
# get all td's from first row in jobs
# note: I'm using [1] since [0] contains headers
for td in jobs[1].find_all('td')[1:]:
    # print stripped text of every td 
    print(td.text.strip())

## 011

Scraper 011 using requests, json and BeautifulSoup.
Goal: find out how much did the state of California collect in property taxes, according to the U.S. Census 2013 Annual Survey of State Government Tax Collections?

*Notes*  
If you look at [this page](https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk), the network tab in Chrome developer tools shows that the data comes from another url, see below.
  
Scraper written December 27th, 2019.

In [ ]:
import requests
import json

# set headers: copy from my browser
headers = {'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'nl-NL,nl;q=0.9,en-US;q=0.8,en;q=0.7',
            'Connection': 'keep-alive',
            'Cookie': 'ak_bmsc=F141B031F58896A0556D9BC25A06B6711743FBB7EB4C00002BF8035EE4547F4E~plOfABtjiiW6tc86p/D+M3clisI2pBwS0MDroFF/A+vKjl0uqKh2LuoGS0t0/9TJ/xAQZzn0ghZd5ek4sanPRczK9TK/zH3I7mMzrV3Rq9ISBsSCJAguan+NiyUT0Q5wFGBRery8468yqn7Aughnzz7wI9SkPdHCne+z8WU1sUUmJ+Bb86EYQXozmC5bl2hu03RnRe7UleaGX9Y3y9JoQWPNcXKXgoamytzsX7UkKjc0w=; _ga=GA1.2.1296164313.1577318444; _gid=GA1.2.1812031464.1577318444; s_fid=26D55BF9E0FDB2E5-30BB5F650193B29B; s_getNewRepeat=1577318444088-New; undefined_s=First%20Visit; s_vnum=1582502444099%26vn%3D1; s_invisit=true; s_cc=true; bm_sv=327AF40BF9F873E4BD3486AC06677B88~LnNsp6HKl9IDCQ4wtITBQO4w2eCLZezeRaZ0Y4cke7IqYN4BY/tRn1yK25J+A03i/kikQxBrrHDxWKKbiUtlMEJZuUF1ZfTObS5fx9wdIlskP8KKLJZoD1stSflUVg1YyC7DPUE49L2yex9H+euboN9qb09GMrXFlUPvb7j0lDg=; JSESSIONID=0001Psr7hoaOmtp3CbuBwf2RtK-:17vl840fa; TS01fa88a5=011ba694f2450b38d6b789354b5cff313038224357cf4ce9d92cee4cc8c28b964ec6b0cc1f4ca7697ad57e0add2a0dd76041c6399e; RDS_DADS_PERSIST=!ScS2W5qxyJqY5tGkG9EXQU5bLFTlrkNJZEWxNlqvpw+RIuzLVKHOa/vF0WzMFylv+6oxuqxotYzLt1g=; s_gnr=1577318524900-New; s_sq=cebufactfinderprod%252Ccebucensusglobalprod%3D%2526c.%2526a.%2526activitymap.%2526page%253DAmerican%252520FactFinder%252520-%252520Results%2526link%253DModify%252520Table%2526region%253DtableToolText%2526pageIDType%253D1%2526.activitymap%2526.a%2526.c%2526pid%253DAmerican%252520FactFinder%252520-%252520Results%2526pidt%253D1%2526oid%253Djavascript%25253AtoggleTT%252528%252529%2526ot%253DA',
            'DNT': '1',
            'Host': 'factfinder.census.gov',
            'Referer': 'https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-origin',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36',
            'X-Requested-With': 'XMLHttpRequest',
            'X-Requested-With': 'XMLHttpRequest'}

# set url as found in network tab
url = 'https://factfinder.census.gov/tablerestful/tableServices/renderProductData?renderForMap=f&renderForChart=f&pid=STC_2013_00A2&prodToReplace=STC_2015_00A2&log=t&_ts=595364512240'
# do request
r = requests.get(url, headers=headers)
# load json
data = json.loads(r.text)
# select table
table = data['ProductData']['productDataTable']
# create soup: json table contains html table
soup = BeautifulSoup(table, 'html.parser')
# select propertyTaxes from soup
# skip some elements - td's are used for spacing -_-
propertyTaxes = soup.find('th', {'id': 'r4'}
                         ).next_element.next_element.next_element.next_element.text
# print result
print(propertyTaxes)


## 012

Scraper 012 using tabula-py and pandas; taking a shortcut by [parsing a PDF](https://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/cy10_primary_enplanements.pdf). :)   
Goal: for 2010, get the year-over-year change in enplanements at America's busiest airport  

Scraper written at December 27th, 2019.

In [ ]:
from tabula import read_pdf
import pandas as pd

# where to find this PDF?
urlPDF = 'https://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/cy10_primary_enplanements.pdf'
# read pdf, create dataframe
df = read_pdf(urlPDF, pages='all')
# rename columns (printed df first time to check column headers)
df.rename(columns = {'Unnamed: 0':'Rank',
                     'CY 10\rEnplanements': 'CY 10 Enplanements',
                     'CY 09\rEnplanements': 'CY 09 Enplanements',
                     '%\rChange': '% Change'}, inplace=True)
# sort df descending by value in CY 10 Enplanements column
df.sort_values(by='CY 10 Enplanements', ascending=False)
# since order is descending for CY 10 Enplanements column, 
# the biggest airport is in the first row = [0]; 
# to find the change, I make ints so I can subtract, 2010 minus 2009.
int(df.iloc[0]['CY 10 Enplanements'].replace(',', '')) - int(df.iloc[0]['CY 09 Enplanements'].replace(',',''))

## TO DO 013
??Scraper 013 using tabula-py and pandas. [PDF with bank crime statistics](https://www.fbi.gov/file-repository/stats-services-publications-bank-crime-statistics-2014-bank-crime-statistics-2014) via [this page](https://www.fbi.gov/file-repository/stats-services-publications-bank-crime-statistics-2014-bank-crime-statistics-2014/view).    

Goal: get the number of armored carrier bank robberies recorded by the FBI in 2014.  

## 014

Scraper 014 using requests, beautifulsoup and pandas.  
Goal: get the number of workplace fatalities at reported to the federal and state OSHA in the latest fiscal year.

Written December 27th, 2019

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# set base url
base = 'https://www.osha.gov'
# found archive url
archive = 'https://www.osha.gov/fatalities/reports/archive'
# get archive, make soup, search for links
r = requests.get(archive)
soup = BeautifulSoup(r.content, 'html.parser')
# create empty list called links
links = []
# for every link found in the soup
for a in soup.find_all('a'):
    try:
        # see if the href of said link contains 'csv'
        if 'csv' in a['href']:
            # if so, append to list
            links.append(a['href'])
        # if not, pass
        else:
            pass
    # if no href found, also pass
    except:
        pass

# get latest csv = first item from list
csvUrl = base + links[0]
df = pd.read_csv(csvUrl)
# print the length (no of rows) of the dataframe, when the dataframe 
# only contains rows where Fatality or Catastrophe column == 'Fatality'
print(len(df[df['Fatality or Catastrophe '] == 'Fatality']))

## TO DO 15 - 94

## 095
Goal: since 2002, the most commonly occurring winning number in New York's Lottery Mega Millions.  
Using: requests and counter.  
Written December 27th, 2019.  

In [ ]:
import requests
from collections import Counter

# create counter
counter = Counter()

# parse json data
numbers = requests.get('https://data.ny.gov/resource/5xaw-6ayf.json').json()

# for every number, split + add to counter
for n in numbers:
    counter.update(n['winning_numbers'].split(' '))

# print most common number + number of times it appeared in dataset
print(counter.most_common()[0])

## 096
Goal: get the number of scheduled arguments according to the most recent U.S. Supreme Court argument calendar.  
Using: requests, beautifulsoup, datetime, and regex.  
Written: December 27th, 2019.

In [ ]:
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup

today = datetime.today()

# request url
url = 'https://www.supremecourt.gov/oral_arguments/calendarsandlists.aspx'
r = requests.get(url)
# make soup, select all links to pdf called argument_calendar
soup = BeautifulSoup(r.content, 'html.parser')
pdfs = soup.find_all('a', href=re.compile('argument_calendar'))
# create empty list of sessions
sessions = []
# for every pdf in pdfs
for p in pdfs:
    # go to parent, and select the text
    session = p.parent.text.strip().replace('Session Beginning ','')
    # clean text up some more
    ses = re.search('(^.*[0-9]{4})\s+\(', session).group(1)
    # use datetime to convert strings to datetime objects
    sesDate = datetime.strptime(ses, '%B %d, %Y')
    # check if sesDate is after today
    if sesDate > today:
        # if so, add sesDate to list sessions
        sessions.append(sesDate)
    # if not, pass
    else:
        pass

# print length of sessions list = planned sessions in the future
print(len(sessions))


## 097
Goal: The New York school with the highest rate of religious exemptions to vaccinations
Using: requests, operator.  
Written: December 27th, 2019.

In [ ]:
import requests
from operator import itemgetter

url = 'https://health.data.ny.gov/resource/5pme-xbs5.json'
data = requests.get(url).json()
# create subset: only if period contains 2019
p1819 = [r for r in data if '2019' in r['report_period']]
# from subset p1819, get all percentreligiousexemptions
# print schoolname of school with max percentreligiousexemptions
max(p1819, key=itemgetter('percentreligiousexemptions'))['schoolname']

In [ ]:
# Alternative: sort p1819 by percentreligiousexemptions in reverse order,
# print schoolname of first row =  [0]['schoolname']
sorted(p1819, key = lambda i: i['percentreligiousexemptions'],reverse=True)[0]['schoolname']

## 098
Using: beautifulsoup en requests.  
Goal: get pct change population in Detroit, Michigan.  
Written: December 27th, 2019.  

In [ ]:
import requests
from bs4 import BeautifulSoup

# go to census url for Detroit
url = 'https://www.census.gov/quickfacts/fact/table/detroitcitymichigan,MI/PST045218'
# get page
r = requests.get(url)
# create soup
soup = BeautifulSoup(r.content, 'html.parser')
# find population percent change 
findPct = soup.find('span', string='Population, percent change - April 1, 2010 (estimates base) to July 1, 2018,  (V2018)')
# get percentage
pct = findPct.next_element.next_element.next_element.next_element.text.strip()
print(pct)

## TO DO 099

## TO DO 100
Using:   
Goals: get the California city whose city manager earns the most total wage per population of its city in 2012.   
Written December 27th, 2019.  

## 101
Goal: get the number of women currently serving in the U.S. Congress. Since the Sunlight Foundation is no longer running an API, I opted for the govtrack.us site.   
Using: requests + json.  
Written December 27th, 2019.   

*Note:  
I found the exact location of the number of females,   
by looking into the json-file using pprint.   
Here's what I did:  
Step 1: pprint.pprint(data)  
Step 2: locate the info you want, in this case it's in 'options'  
Step 3: pprint.pprint(data['options'])  
Step 4: locate the info you want, in this case it's in '6'  
Step 5: pprint.pprint(data['options'][6])  
... etcetera*

In [ ]:
import requests
import json
import pprint

# url found while looking at network tab in browser
url = 'https://www.govtrack.us/congress/members/current?sort=sortname&page=1&faceting=false&allow_redirect=false&do_search=1'
# do requests
r = requests.get(url)
# load json
data = json.loads(r.text)

# number of females in congress
data['options'][6][2][2][2]